In [1]:
import pandas as pd
import requests as rq
import comtradeapicall as comtradeapicall
from datetime import datetime, timedelta
import os
import time

In [2]:
# Get the current working directory
current_directory = os.getcwd()

In [4]:
subscription_key = '' # FREE comtrade api subscription key (from comtradedeveloper.un.org)


In [5]:
##############################################################################################

In [6]:
code = pd.read_csv("HS_CODE_CAT.csv",dtype={'CODE': str})
reporter = pd.read_csv("reporterAreas.csv",dtype={'CODE': str})
partner = pd.read_csv("partnerAreas.csv",dtype={'CODE': str})

In [7]:
code.dropna(how = 'all', axis=1, inplace = True)
code.dropna(how = 'all', axis=0, inplace = True)

In [8]:
#################################################################################################

#### Dates

In [9]:
from datetime import datetime

def data_gen(start_date):
    # Convert the input date to a datetime object
    start_date = datetime.strptime(start_date, '%Y%m')
    
    # Initialize an empty list to store the date strings
    date_strings = []
    
    while len(date_strings) < 12:
        date_strings.append(start_date.strftime('%Y%m'))
        # Increment the month
        if start_date.month == 12:
            # If the current month is December, increment the year and set the month to January
            start_date = start_date.replace(year=start_date.year + 1, month=1)
        else:
            start_date = start_date.replace(month=start_date.month + 1)
    
    # Join the date strings into a single string separated by commas
    result = ','.join(date_strings)

    return result

# Example usage
start_date = '202101'
dates = data_gen(start_date)
print(dates)



202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112


In [10]:
def annual_gen(start_date, end_date):
    start_date = datetime.strptime(start_date, '%Y%m')
    end_date = datetime.strptime(end_date, '%Y%m')
    
    date_str = []
    while start_date <= end_date:
        date_str.append(start_date.strftime('%Y%m'))
        start_date = start_date.replace(year=start_date.year + 1)
    return date_str
result = annual_gen('200001','202309')
print(result)

['200001', '200101', '200201', '200301', '200401', '200501', '200601', '200701', '200801', '200901', '201001', '201101', '201201', '201301', '201401', '201501', '201601', '201701', '201801', '201901', '202001', '202101', '202201', '202301']


### Reporter and Parter

In [11]:
# Assuming you have a DataFrame 'df' with columns 'id' and 'text'

def search_entries_by_initial_character(df, initial_character):
    matching_entries = df[df['text'].str.startswith(initial_character)]
    return matching_entries[['id', 'text']]



initial_character = 'Spa'
entry = search_entries_by_initial_character(reporter, initial_character)
entry

,id,text
217,724,Spain


#### Code

In [12]:
compressed_string = ','.join(map(str, code['CODE']))

In [13]:
### 250k call 

In [14]:
#  '842,276,381,251,724'
# flwCode	flwDescription	flwCategory
#-1	Not available or not specified.	
# M	Import	M
# X	Export	X
# RX	Re-export	X
# RM	Re-import	M
# MIP	Import of goods for inward processing	M
# XIP	Export of goods after inward processing	X
# MOP	Import of goods after outward processing	M
# XOP	Export of goods for outward processing	X
# MIF	Import on intra-firm trade	M
# XIF	Export on intra-firm trade	X
# DX	Domestic Export	X
# FM	Foreign Import	M'


In [15]:
mydf_251 = []  # Initialize an empty list to store the API call results

for date in result:
    response = comtradeapicall.getFinalData(
        subscription_key, typeCode='C', freqCode='M', clCode='HS', period=data_gen(date),
        reporterCode='724', cmdCode=compressed_string, flowCode='X', partnerCode=0,
        partner2Code=None, customsCode=None, motCode=None, maxRecords=250000,
        format_output='JSON', aggregateBy=None, breakdownMode='classic', countOnly=None, includeDesc=True
    )
    time.sleep(10)
    mydf_251.append(response)  # Append the API call response to the list

# Now mydf contains the results of the API calls for each date in the result list


In [16]:
merged_df_251 = pd.concat(mydf_251, ignore_index=True)

In [17]:
merged_df_251.to_csv("ESP_EXP_GOODS.csv")

response = comtradeapicall.getFinalData(
    subscription_key, typeCode='S', freqCode='A', clCode='EB10', period='2023,2022,2021',
    reporterCode='842,276,380,250', cmdCode='200', flowCode='X', partnerCode=0,
    partner2Code=None, customsCode=None, motCode=None, maxRecords=250000,
    format_output='JSON', aggregateBy=None, breakdownMode='classic', countOnly=None, includeDesc=True
)